# Input data part


In [1]:
import json
import os
import logging

__file__ = './'
with open(os.path.join(os.path.dirname(__file__), 'extracted_content_directory.json')) as f:
    data = json.load(f)
    
print("data loaded")
print(f"data: {data}")

data loaded
data: {'https://www.seattleu.edu/directory/': 'Faculty & Staff Directory\nExplore the Directory\nSearch by Department\nNext steps\n', 'https://www.seattleu.edu/directory/profiles/': '', 'https://www.seattleu.edu/directory/profiles/nicole-plastino.php': '', 'https://www.seattleu.edu/directory/profiles/keith-mobley-phd-lmhcs-lcmhcs-acs-ncc.php': 'Profile Type:\nAssociate Clinical Professor, Clinical Mental Health Counseling (CMHC)\nContact Information\n', 'https://www.seattleu.edu/directory/profiles/aaron-clarry.php': '', 'https://www.seattleu.edu/directory/profiles/aaron-day-edd.php': '', 'https://www.seattleu.edu/directory/profiles/aaron-morgan.php': '', 'https://www.seattleu.edu/directory/profiles/aaron-m-thornburg-jd-llm-mba-mir.php': 'Profile Type:\nContact Information\nPersonal Bio\nAaron M. Thornburg is a lawyer, professor, real estate broker and investor, business owner, board member, 501c3 founder, and unpaid intern at his wife’s Seattle-based company, Skinny Dipped.

In [4]:
data.get('https://www.seattleu.edu/')

"Whole Person Education\nHigher-Powered Learning\nAt Seattle University, we empower you to realize your full potential through an inclusive and holistic educational experience grounded in universal values.\nMeaningful Careers\nOur Jesuit education and deep ties to the best companies in Seattle prepare you for success in your career and community—wherever life takes you.\nExplore Programs\nSeattle University doesn’t just prepare you for a constantly changing world—we help you shape it for the better. Find the programs that ignite your passion and help you make a difference.\nLiving Our Values\nWe’re on a mission to empower you to create a more just, equitable and sustainable world through doing the work you love.\nTanaporn Sittisawassakul, ’18\n“Through experience, I have learned that the Jesuits are really really great at education and especially education of the whole self, critical thinking. Not just mind, but body and spirit.”\nStrategic Communications\nShannon Blott (She/Her), ’24\

In [2]:
import re

def clean_text(text):

    text_no_punct = re.sub(r'[^\w\s]', '', text)
    normalized_text = re.sub(r'\s+', ' ', text_no_punct).strip()
    cleaned_text = normalized_text.lower()
    return cleaned_text

# print(clean_text(data.get('https://www.seattleu.edu/')))

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env.local file
load_dotenv(dotenv_path='.env.local')

# Get Pinecone API key
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENV')


In [4]:
from pinecone import Pinecone, ServerlessSpec
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core import Document
from llama_index.vector_stores.pinecone import PineconeVectorStore
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding

/Users/rohitnagotkar/su_rag/rag_env/lib/python3.13/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'su-rag-pipeline'
dimension = 384  # Embedding dimension for all-MiniLM-L6-v2
print(pc.list_indexes())
# Create the Pinecone index if it doesn't exist.
# if index_name not in pc.list_indexes():
#     pc.create_index(name=index_name, dimension=dimension, metric="cosine", spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ) )
#     print(f"Created pc index: {index_name}")
# else:
#     print(f"Pinecone index '{index_name}' already exists.")

# pinecone_index = pc.Index(index_name)


{'indexes': [{'dimension': 384,
              'host': 'su-rag-pipeline-7gxtmye.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'su-rag-pipeline',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [9]:
# Process each value in the dictionary through clean_text
cleaned_data = {}

for key, value in data.items():
    cleaned_data[key] = clean_text(value)
    
# Convert the dictionary to a list of dictionaries with 'id' and 'text' fields
data_list = []
for key, value in cleaned_data.items():
    if len(value) > 0:
        data_list.append({
            'id': key,  # URL as the id
            'text': value  # Cleaned text as the value
        })

# Print the first item to verify
print(f"First item in list: {data_list[40]}")
print(f"Total items in list: {len(data_list)}")

First item in list: {'id': 'https://www.seattleu.edu/directory/profiles/angela-siple-phd.php', 'text': 'research interest biography dr siple has a bs a ms and a phd from the university of central florida she has been on faculty at seattle university since 2016 next steps'}
Total items in list: 750


In [34]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

  Using cached llama_index_core-0.12.30-py3-none-any.whl.metadata (2.6 kB)
Using cached llama_index_core-0.12.30-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.68.post1
    Uninstalling llama-index-core-0.10.68.post1:
      Successfully uninstalled llama-index-core-0.10.68.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-vector-stores-pinecone 0.1.2 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.12.30 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 4.0.2
    Uninstalling sentence-transformers-

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Initialize the all-MiniLM-L6-v2 embedding model
embedding_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")
# Generate embeddings for all texts in data_list
embeddings = []
for item in data_list:
    # Get the embedding for each item's text
    embedding = embedding_model.get_text_embedding(item['text'])
    # Format in the way Pinecone expects
    embeddings.append({
        'vector_type': 'dense',
        'values': embedding
    })

# Print the number of generated embeddings
print(f"Generated {len(embeddings)} embeddings")
print(embeddings[0])

Generated 750 embeddings
{'vector_type': 'dense', 'values': [0.01781201735138893, -0.052444230765104294, -0.02385731041431427, -0.025169488042593002, 0.013437719084322453, -0.013499551452696323, -0.07173994183540344, -0.036151885986328125, -0.0016689446056261659, -0.004126929212361574, 0.06694492697715759, 0.044995199888944626, -0.056467246264219284, -0.01170220784842968, -0.07356655597686768, -0.03884848579764366, -0.021581709384918213, 0.045837268233299255, 0.06071201339364052, -0.049823153764009476, 0.04726260527968407, 0.09191549569368362, 0.0371270477771759, -0.07424766570329666, -0.0590190552175045, -0.039308711886405945, -0.020188551396131516, -0.10185644030570984, -0.03551057353615761, 0.04271417111158371, 0.05386785790324211, 0.0541742742061615, 0.05911559984087944, 0.07259313017129898, 0.02233251929283142, 0.031694576144218445, -0.004826351068913937, 0.0914146900177002, 0.07295315712690353, 0.053709812462329865, -0.0863633006811142, 0.012572119943797588, 0.057135894894599915,

In [11]:
pc.describe_index(index_name).status['ready']

True

In [12]:
index = pc.Index(index_name)
BATCH_SIZE = 25  # Smaller batch size to stay under 2MB limit
MAX_METADATA_TEXT_LENGTH = 15000  # Conservative limit to stay under 40KB


# Split vectors into batches
def batch_vectors():
    batch = []
    for d, e in zip(data_list, embeddings):
        truncated_text = d['text'][:MAX_METADATA_TEXT_LENGTH]
        vector = {
            "id": d['id'],
            "values": e['values'],
            "metadata": {'text': truncated_text, 'url': d['id']}
        }
        batch.append(vector)
        
        if len(batch) >= BATCH_SIZE:
            yield batch
            batch = []
    
    if batch:  # Don't forget the last batch
        yield batch

# Upload vectors in batches
for i, vector_batch in enumerate(batch_vectors(), 1):
    try:
        index.upsert(
            vectors=vector_batch,
            namespace="poc_rag"
        )
        print(f"Successfully uploaded batch {i} ({len(vector_batch)} vectors)")
    except Exception as e:
        print(f"Error uploading batch {i}: {e}")

Successfully uploaded batch 1 (25 vectors)
Successfully uploaded batch 2 (25 vectors)
Successfully uploaded batch 3 (25 vectors)
Successfully uploaded batch 4 (25 vectors)
Successfully uploaded batch 5 (25 vectors)
Successfully uploaded batch 6 (25 vectors)
Successfully uploaded batch 7 (25 vectors)
Successfully uploaded batch 8 (25 vectors)
Successfully uploaded batch 9 (25 vectors)
Successfully uploaded batch 10 (25 vectors)
Successfully uploaded batch 11 (25 vectors)
Successfully uploaded batch 12 (25 vectors)
Successfully uploaded batch 13 (25 vectors)
Successfully uploaded batch 14 (25 vectors)
Successfully uploaded batch 15 (25 vectors)
Successfully uploaded batch 16 (25 vectors)
Successfully uploaded batch 17 (25 vectors)
Successfully uploaded batch 18 (25 vectors)
Successfully uploaded batch 19 (25 vectors)
Successfully uploaded batch 20 (25 vectors)
Successfully uploaded batch 21 (25 vectors)
Successfully uploaded batch 22 (25 vectors)
Successfully uploaded batch 23 (25 vector

In [13]:
query = "What professors are there who teach computer science?"

embedding_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Generate the embedding for your query.
embedding = embedding_model.get_text_embedding(query)
print(f"Query embedding: {embedding}")
results = index.query(
    namespace="poc_rag",
    vector=embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

Query embedding: [0.004291863646358252, -0.06311384588479996, -0.056935619562864304, -0.001609654282219708, -0.04289226979017258, -0.07217592000961304, 0.012392883189022541, -0.03403438627719879, 0.02466694265604019, 0.05030246451497078, -0.04619284346699715, -0.007921777665615082, -0.010229332372546196, -0.017766961827874184, -0.020970087498426437, 0.05789121985435486, -0.08497780561447144, -0.04377400502562523, 0.057566314935684204, -0.10221965610980988, -0.021711573004722595, 0.010279382579028606, -0.003820965299382806, -0.07539287954568863, 0.017805200070142746, 0.014028993435204029, -0.0028147923294454813, -0.0814439207315445, -0.02948753349483013, -0.033870697021484375, -0.10442837327718735, -0.012087007984519005, 0.03508754447102547, 0.07398921251296997, 0.011253171600401402, 0.035451386123895645, 0.05113215371966362, 0.028641344979405403, -0.000422130455262959, 0.010078050196170807, -0.04454239830374718, 0.08618994802236557, 0.0381036251783371, 0.01042013056576252, 0.0247913673